In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow import keras

# Importing specific modules from libraries
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Setting numpy print options to display floating point numbers up to 3 decimal points
np.set_printoptions(precision=3, suppress=True)


<IPython.core.display.Javascript object>

### 1. Load the datasets

##### leakage dataset train [m].csv
##### leakage dataset validation 1000.csv

and store the data as NumPy-Arrays
X train, Y train
X validation, Y validation.

In [3]:
# Loading the data from CSV files

# Reading the first dataset from a CSV file named "leakage_dataset_train_100.csv" and storing it in a pandas dataframe named df_100
df_100 = pd.read_csv("Data/leakage_dataset_train_100.csv", header=0)

# Reading the second dataset from a CSV file named "leakage_dataset_train_1000.csv" and storing it in a pandas dataframe named df_1000
df_1000 = pd.read_csv("Data/leakage_dataset_train_1000.csv", header=0)

# Reading the validation dataset from a CSV file named "leakage_dataset_validation_1000.csv" and storing it in a pandas dataframe named df_val
df_val = pd.read_csv("Data/leakage_dataset_validation_1000.csv", header=0)


<IPython.core.display.Javascript object>

In [4]:
# Load the features and label from dataframe and convert it to numpy array

# List the columns for the features and label for each dataframe
feature_cols = ['mfc1', 'mfc2', 'mfc3', 'mfc4']
label_cols = ['y1', 'y2']

# Load the 100-row training set and convert the features and label to numpy arrays
train_100 = df_100[feature_cols + label_cols].to_numpy()
X_train_100, y_train_100 = train_100[:, :4], train_100[:, 4:]

# Load the 1000-row training set and convert the features and label to numpy arrays
train_1000 = df_1000[feature_cols + label_cols].to_numpy()
X_train_1000, y_train_1000 = train_1000[:, :4], train_1000[:, 4:]

# Load the validation set and convert the features and label to numpy arrays
val = df_val[feature_cols + label_cols].to_numpy()
X_val, y_val = val[:, :4], val[:, 4:]


<IPython.core.display.Javascript object>

In [5]:
# # Calculate the mean and standard deviation of x1, x2, and x3 in the training data
# x1_mean = np.mean(X_train_1000[:, 0])
# x2_mean = np.mean(X_train_1000[:, 1])
# x3_mean = np.mean(X_train_1000[:, 2])
# x4_mean = np.mean(X_train_1000[:, 3])

# x1_std = np.std(X_train_1000[:, 0])
# x2_std = np.std(X_train_1000[:, 1])
# x3_std = np.std(X_train_1000[:, 2])
# x4_std = np.std(X_train_1000[:, 3])

<IPython.core.display.Javascript object>

In [6]:
# # Subtract the mean from x1, x2, and x3 in both the training and validation/test data
# X_train_1000[:, 0] = (X_train_1000[:, 0] - x1_mean) / x1_std
# X_train_1000[:, 1] = (X_train_1000[:, 1] - x2_mean) / x2_std
# X_train_1000[:, 2] = (X_train_1000[:, 2] - x3_mean) / x3_std
# X_train_1000[:, 3] = (X_train_1000[:, 3] - x4_mean) / x4_std

# X_train_100[:, 0] = (X_train_100[:, 0] - x1_mean) / x1_std
# X_train_100[:, 1] = (X_train_100[:, 1] - x2_mean) / x2_std
# X_train_100[:, 2] = (X_train_100[:, 2] - x3_mean) / x3_std
# X_train_100[:, 3] = (X_train_100[:, 3] - x4_mean) / x4_std

# X_val[:, 0] = (X_val[:, 0] - x1_mean) / x1_std
# X_val[:, 1] = (X_val[:, 1] - x2_mean) / x2_std
# X_val[:, 2] = (X_val[:, 2] - x3_mean) / x3_std
# X_val[:, 3] = (X_val[:, 3] - x4_mean) / x4_std

<IPython.core.display.Javascript object>

In [7]:
# # Calculate the mean and standard deviation of x1, x2, and x3 in the training data
# x_mean = np.mean(X_train_1000, axis=0)
# x_std = np.std(X_train_1000, axis=0)

# # Standardize the data
# X_train_1000 = (X_train_1000 - x_mean) / x_std
# X_train_100 = (X_train_100 - x_mean) / x_std
# X_val = (X_val - x_mean) / x_std

# # Normalize the data
# X_train_1000 = X_train_1000 / np.sum(X_train_1000, axis=1)[:, np.newaxis]
# X_train_100 = X_train_100 / np.sum(X_train_100, axis=1)[:, np.newaxis]
# X_val = X_val / np.sum(X_val, axis=1)[:, np.newaxis]

<IPython.core.display.Javascript object>

For that purpose, you have training data (xi, yi)
mi=1 available. Unfortunately,
the fourth sensor MFC4 had a slight malfunction at the time when the training data were produced. As a consequence, the values x4 are on average
higher than their counterparts x1, x2 and x3. Be aware that the same effect
is not present in the validation and test data.

In [8]:
# Preprocess the training data to account for the issue with the fourth sensor MFC4
X_train_100[:, 3] = X_train_100[:, 3] * (
    np.mean(X_train_100[:, :3], axis=1) / np.mean(X_train_100[:, 3])
)
X_train_1000[:, 3] = X_train_1000[:, 3] * (
    np.mean(X_train_1000[:, :3], axis=1) / np.mean(X_train_1000[:, 3])
)

<IPython.core.display.Javascript object>

### 2. Train a standard fully connected neural network. Hyperparameters

like number of epochs, batch size, optimizer, learning rate, depth and
width of the network, activation and loss functions – all up to you. You
can use a framework like KerasTuner to optimize hyperparameters, or
simple for-loops if you prefer that. Anyway, you should use the validation data in order to evaluate and compare candidate hyperparameter
configurations. Save the model as model standard [m].h5.

In [9]:
def standard_model(hp):
    model = Sequential()
    model.add(
        Dense(
            units=hp.Int("units_1", min_value=32, max_value=512, step=32),
            input_shape=(4,),
            activation=hp.Choice("activation_1", values=["relu", "tanh", "sigmoid"]),
        )
    )

    model.add(
        Dense(
            units=hp.Int(f"units_2", min_value=16, max_value=128, step=16),
            activation=hp.Choice(
                f"activation_2",
                values=["relu", "tanh", "sigmoid"],
            ),
        )
    )

    model.add(Dense(2, activation="softmax"))

    # Set optimizer and learning rate
    optimizer = hp.Choice("optimizer", values=["adam", "adagrad", "nadam", "rmsprop"])
    learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "adagrad":
        optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )

    return model

<IPython.core.display.Javascript object>

In [10]:
# def with_all_standard_model(hp):
#     model = Sequential()
#     model.add(
#         Dense(
#             units=hp.Int("units_1", min_value=32, max_value=512, step=32),
#             input_shape=(4,),
#             activation=hp.Choice("activation_1", values=["relu", "tanh", "sigmoid"]),
#         )
#     )

#     model.add(Dropout(hp.Float("dropout_1", min_value=0.2, max_value=0.5, step=0.1)))
#     model.add(BatchNormalization())

#     model.add(
#         Dense(
#             units=hp.Int(f"units_2", min_value=16, max_value=128, step=16),
#             activation=hp.Choice(
#                 f"activation_2",
#                 values=["relu", "tanh", "sigmoid"],
#             ),
#         )
#     )

#     model.add(Dropout(hp.Float(f"dropout_2", min_value=0.2, max_value=0.5, step=0.1)))
#     model.add(BatchNormalization())

#     model.add(Dense(2, activation="softmax"))

#     # Set optimizer and learning rate
#     optimizer = hp.Choice("optimizer", values=["adam", "adagrad", "nadam", "rmsprop"])
#     learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])

#     if optimizer == "adam":
#         optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
#     elif optimizer == "adagrad":
#         optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
#     elif optimizer == "rmsprop":
#         optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
#     else:
#         optimizer = keras.optimizers.Nadam(learning_rate=learning_rate)

#     model.compile(
#         optimizer=optimizer,
#         loss="mean_squared_error",
#         metrics=["mean_absolute_error"],
#     )

#     # Add regularization
#     if hp.Boolean("use_regularization"):
#         l1_reg = hp.Float("l1_reg", min_value=1e-6, max_value=1e-2, sampling="log")
#         l2_reg = hp.Float("l2_reg", min_value=1e-6, max_value=1e-2, sampling="log")
#         model.add(keras.layers.ActivityRegularization(l1=l1_reg, l2=l2_reg))

#     return model

<IPython.core.display.Javascript object>

In [11]:
tuner_100 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_100",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_100.search(
    X_train_100,
    y_train_100,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    #     verbose=1,
)

Trial 254 Complete [00h 00m 06s]
val_loss: 0.570703387260437

Best val_loss So Far: 0.42471805214881897
Total elapsed time: 00h 15m 22s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [12]:
best_hps_100 = tuner_100.get_best_hyperparameters(num_trials=1)[0]
model_100 = tuner_100.hypermodel.build(best_hps_100)

<IPython.core.display.Javascript object>

In [13]:
# Train the model on the training set
history_100 = model_100.fit(
    X_train_100,
    y_train_100,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

Epoch 1/100
4/4 [==============================] - 1s 67ms/step - loss: 0.5626 - mean_absolute_error: 0.6136 - val_loss: 0.5205 - val_mean_absolute_error: 0.5889
Epoch 2/100
4/4 [==============================] - 0s 27ms/step - loss: 0.5307 - mean_absolute_error: 0.5975 - val_loss: 0.4865 - val_mean_absolute_error: 0.5719
Epoch 3/100
4/4 [==============================] - 0s 23ms/step - loss: 0.5041 - mean_absolute_error: 0.5789 - val_loss: 0.4681 - val_mean_absolute_error: 0.5580
Epoch 4/100
4/4 [==============================] - 0s 23ms/step - loss: 0.4826 - mean_absolute_error: 0.5714 - val_loss: 0.4527 - val_mean_absolute_error: 0.5583
Epoch 5/100
4/4 [==============================] - 0s 22ms/step - loss: 0.4704 - mean_absolute_error: 0.5608 - val_loss: 0.4359 - val_mean_absolute_error: 0.5425
Epoch 6/100
4/4 [==============================] - 0s 21ms/step - loss: 0.4584 - mean_absolute_error: 0.5565 - val_loss: 0.4303 - val_mean_absolute_error: 0.5393
Epoch 7/100
4/4 [===========

<IPython.core.display.Javascript object>

In [14]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_100.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_100.save("models/model_standard_100.h5")

32/32 [==============================] - 0s 936us/step - loss: 0.4315 - mean_absolute_error: 0.5370
Validation loss: 0.432
Validation accuracy: 0.537


<IPython.core.display.Javascript object>

In [15]:
tuner_1000 = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_1000",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_1000.search(
    X_train_1000,
    y_train_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
#     verbose=1,
)


Trial 254 Complete [00h 00m 14s]
val_loss: 0.4246198534965515

Best val_loss So Far: 0.4234359562397003
Total elapsed time: 00h 17m 43s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [16]:
best_hps_1000 = tuner_1000.get_best_hyperparameters(num_trials=1)[0]
model_1000 = tuner_1000.hypermodel.build(best_hps_1000)

<IPython.core.display.Javascript object>

In [17]:
# Train the model on the training set
history_1000 = model_1000.fit(
    X_train_1000,
    y_train_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

Epoch 1/100
32/32 [==============================] - 1s 8ms/step - loss: 0.4572 - mean_absolute_error: 0.5592 - val_loss: 0.4265 - val_mean_absolute_error: 0.5356
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4283 - mean_absolute_error: 0.5398 - val_loss: 0.4256 - val_mean_absolute_error: 0.5364
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4280 - mean_absolute_error: 0.5400 - val_loss: 0.4256 - val_mean_absolute_error: 0.5365
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4279 - mean_absolute_error: 0.5396 - val_loss: 0.4343 - val_mean_absolute_error: 0.5425
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4281 - mean_absolute_error: 0.5399 - val_loss: 0.4271 - val_mean_absolute_error: 0.5368
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.4276 - mean_absolute_error: 0.5395 - val_loss: 0.4247 - val_mean_absolute_error: 0.5359
Epoch 7/100
32/32 [===

<IPython.core.display.Javascript object>

In [18]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_1000.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_1000.save("models/model_standard_1000.h5")

32/32 [==============================] - 0s 1ms/step - loss: 0.4258 - mean_absolute_error: 0.5362
Validation loss: 0.426
Validation accuracy: 0.536


<IPython.core.display.Javascript object>

### 3. Augment the dataset. 

Each training example (x, y) can be complemented by seven additional virtual examples as illustrated in Figure 1.
Clockwise rotations and flips on input data x are obtained as described
above. A clockwise rotation on output data y is obtained through
y90 = (y2, −y1) and a flip along the vertical axis on the same data is
represented by yf lipped = (−y1, y2). All other operations (rotation angles and associated flips) can be computed by subsequent application
of these two operations.

In [19]:
def augment_data(x_train, y_train):
    # initialize empty lists for augmented data and labels
    x_train_aug = []
    y_train_aug = []

    for i in range(len(x_train)):
        x_train_aug.append(x_train[i])
        y_train_aug.append(y_train[i])

        # 90 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][1], x_train[i][0], -x_train[i][3], x_train[i][2]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][1], -y_train[i][0]])))

        # 180 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][2], -x_train[i][3], x_train[i][0], x_train[i][1]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][0], -y_train[i][1]])))

        # 270 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][3], -x_train[i][2], x_train[i][1], -x_train[i][0]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][1], y_train[i][0]])))

        # vertical flip
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][2], x_train[i][3], x_train[i][0], x_train[i][1]])
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][0], y_train[i][1]])))

        # horizontal flip + 90 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array(
                    [-x_train[i][1], -x_train[i][0], -x_train[i][3], -x_train[i][2]]
                )
            )
        )
        y_train_aug.append(np.flip(np.array([-y_train[i][1], y_train[i][0]])))

        # horizontal flip + 180 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([-x_train[i][2], -x_train[i][1], x_train[i][0], x_train[i][3]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][0], y_train[i][1]])))

        # horizontal flip + 270 degree clockwise rotation
        x_train_aug.append(
            np.flip(
                np.array([x_train[i][3], x_train[i][2], -x_train[i][1], x_train[i][0]])
            )
        )
        y_train_aug.append(np.flip(np.array([y_train[i][1], -y_train[i][0]])))
    return x_train_aug, y_train_aug

<IPython.core.display.Javascript object>

In [20]:
# Augment the training data with the augment_data function for the X_train_100 and y_train_100 data
x_train_augmented, y_train_augmented = augment_data(X_train_100, y_train_100)
x_train_augmented_1000, y_train_augmented_1000 = augment_data(
    X_train_1000, y_train_1000
)

# Convert the augmented data and labels to numpy arrays
X_train_augmented = np.array(x_train_augmented)
y_train_augmented = np.array(y_train_augmented)
X_train_augmented_1000 = np.array(x_train_augmented_1000)
y_train_augmented_1000 = np.array(y_train_augmented_1000)

<IPython.core.display.Javascript object>

### 4. Repeat 2. 
on the augmented dataset. Save this model as model standard [m] augmented.h5.

In [21]:
tuner_100_aug = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_aug_100",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_100_aug.search(
    X_train_augmented,
    y_train_augmented,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    #     verbose=1,
)

Trial 254 Complete [00h 00m 17s]
val_loss: 0.5613260269165039

Best val_loss So Far: 0.47030162811279297
Total elapsed time: 00h 15m 36s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [22]:
best_hps_aug_100 = tuner_100_aug.get_best_hyperparameters(num_trials=1)[0]
model_aug_100 = tuner_100_aug.hypermodel.build(best_hps_aug_100)

<IPython.core.display.Javascript object>

In [23]:
# Train the model on the training set
history_aug_100 = model_aug_100.fit(
    X_train_augmented,
    y_train_augmented,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

Epoch 1/100
25/25 [==============================] - 1s 10ms/step - loss: 0.5794 - mean_absolute_error: 0.6217 - val_loss: 0.5620 - val_mean_absolute_error: 0.6123
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5561 - mean_absolute_error: 0.6071 - val_loss: 0.5705 - val_mean_absolute_error: 0.6192
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5535 - mean_absolute_error: 0.6059 - val_loss: 0.5834 - val_mean_absolute_error: 0.6241
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5339 - mean_absolute_error: 0.5963 - val_loss: 0.5740 - val_mean_absolute_error: 0.6185
Epoch 5/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5308 - mean_absolute_error: 0.5937 - val_loss: 0.5662 - val_mean_absolute_error: 0.6138
Epoch 6/100
25/25 [==============================] - 0s 4ms/step - loss: 0.5254 - mean_absolute_error: 0.5895 - val_loss: 0.5649 - val_mean_absolute_error: 0.6128
Epoch 7/100
25/25 [==

<IPython.core.display.Javascript object>

In [24]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_aug_100.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_aug_100.save("models/model_standard_100_augmented.h5")

32/32 [==============================] - 0s 1ms/step - loss: 0.5436 - mean_absolute_error: 0.6018
Validation loss: 0.544
Validation accuracy: 0.602


<IPython.core.display.Javascript object>

In [25]:
tuner_1000_aug = kt.Hyperband(
    hypermodel=standard_model,
    objective="val_loss",
    max_epochs=100,
    factor=3,
    overwrite=True,
    directory="my_dir",
    project_name="standard_model_aug_1000",
)
# Search for the best hyperparameters
early_stopping = EarlyStopping(monitor="val_loss", patience=10)
tuner_1000_aug.search(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    #     verbose=1,
)

Trial 254 Complete [00h 00m 32s]
val_loss: 0.5588197112083435

Best val_loss So Far: 0.44556111097335815
Total elapsed time: 00h 28m 10s
INFO:tensorflow:Oracle triggered exit


<IPython.core.display.Javascript object>

In [26]:
best_hps_aug_1000 = tuner_1000_aug.get_best_hyperparameters(num_trials=1)[0]
model_aug_1000 = tuner_1000_aug.hypermodel.build(best_hps_aug_1000)

<IPython.core.display.Javascript object>

In [27]:
# Train the model on the training set
history_aug_1000 = model_aug_1000.fit(
    X_train_augmented_1000,
    y_train_augmented_1000,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5444 - mean_absolute_error: 0.6042 - val_loss: 0.5239 - val_mean_absolute_error: 0.5877
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5180 - mean_absolute_error: 0.5906 - val_loss: 0.5111 - val_mean_absolute_error: 0.5826
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5092 - mean_absolute_error: 0.5857 - val_loss: 0.5737 - val_mean_absolute_error: 0.6164
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.5025 - mean_absolute_error: 0.5818 - val_loss: 0.4996 - val_mean_absolute_error: 0.5800
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4956 - mean_absolute_error: 0.5785 - val_loss: 0.5976 - val_mean_absolute_error: 0.6236
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4922 - mean_absolute_error: 0.5763 - val_loss: 0.4655 - val_mean_absolute_error: 0.5690
Epoch 7/10

<IPython.core.display.Javascript object>

In [28]:
# Evaluate the model on the validation dataset
val_loss, val_acc = model_aug_1000.evaluate(X_val, y_val)
print("Validation loss: {:.3f}".format(val_loss))
print("Validation accuracy: {:.3f}".format(val_acc))


# Save the model
model_aug_100.save("models/model_standard_1000_augmented.h5")

32/32 [==============================] - 0s 1ms/step - loss: 0.6276 - mean_absolute_error: 0.6574
Validation loss: 0.628
Validation accuracy: 0.657


<IPython.core.display.Javascript object>

In [29]:
model_aug_1000.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 480)               2400      
                                                                 
 dense_4 (Dense)             (None, 64)                30784     
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 33,314
Trainable params: 33,314
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [30]:
best_hps_aug_1000.values

{'units_1': 480,
 'activation_1': 'relu',
 'units_2': 64,
 'activation_2': 'tanh',
 'optimizer': 'rmsprop',
 'learning_rate': 0.01,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

<IPython.core.display.Javascript object>

In [32]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(480, activation='relu', input_shape=(4,)))
model.add(tf.keras.layers.Dense(64, activation='tanh'))
model.add(tf.keras.layers.Dense(2))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)


<IPython.core.display.Javascript object>

In [34]:
# Train the model
history = model.fit(
    X_train_augmented_1000,
    y_train_augmented_1000,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=100,
    callbacks=[callback],
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.2288 - mean_absolute_error: 0.3724 - val_loss: 0.1878 - val_mean_absolute_error: 0.3355
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1419 - mean_absolute_error: 0.2923 - val_loss: 0.1498 - val_mean_absolute_error: 0.2925
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1196 - mean_absolute_error: 0.2663 - val_loss: 0.1334 - val_mean_absolute_error: 0.2836
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.1078 - mean_absolute_error: 0.2508 - val_loss: 0.1396 - val_mean_absolute_error: 0.2769
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0993 - mean_absolute_error: 0.2401 - val_loss: 0.2309 - val_mean_absolute_error: 0.3901
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.0919 - mean_absolute_error: 0.2291 - val_loss: 0.1340 - val_mean_absolute_error: 0.2951
Epoch 7/10

250/250 [==============================] - 1s 2ms/step - loss: 0.0356 - mean_absolute_error: 0.1310 - val_loss: 0.1680 - val_mean_absolute_error: 0.3183
Epoch 51/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0351 - mean_absolute_error: 0.1312 - val_loss: 0.0975 - val_mean_absolute_error: 0.2289
Epoch 52/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0351 - mean_absolute_error: 0.1311 - val_loss: 0.2354 - val_mean_absolute_error: 0.3628
Epoch 53/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0341 - mean_absolute_error: 0.1289 - val_loss: 0.1080 - val_mean_absolute_error: 0.2527
Epoch 54/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0344 - mean_absolute_error: 0.1293 - val_loss: 0.2072 - val_mean_absolute_error: 0.3304
Epoch 55/100
250/250 [==============================] - 0s 2ms/step - loss: 0.0331 - mean_absolute_error: 0.1282 - val_loss: 0.0758 - val_mean_absolute_error: 0.2141
Epoch 56/100
250/

250/250 [==============================] - 0s 2ms/step - loss: 0.0279 - mean_absolute_error: 0.1161 - val_loss: 0.1318 - val_mean_absolute_error: 0.2656
Epoch 100/100
250/250 [==============================] - 1s 2ms/step - loss: 0.0290 - mean_absolute_error: 0.1178 - val_loss: 0.1470 - val_mean_absolute_error: 0.2799


<IPython.core.display.Javascript object>

In [37]:
val_loss, val_acc = model_aug_1000.evaluate(X_val, y_val)

32/32 [==============================] - 0s 1ms/step - loss: 0.6276 - mean_absolute_error: 0.6574


<IPython.core.display.Javascript object>

In [36]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 480)               2400      
                                                                 
 dense_7 (Dense)             (None, 64)                30784     
                                                                 
 dense_8 (Dense)             (None, 2)                 130       
                                                                 
Total params: 33,314
Trainable params: 33,314
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model with best hyperparameters found by the tuner
history_extra = model_100.fit(
    X_train_100,
    y_train_100,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, ModelCheckpoint("models/best_model.h5", save_best_only=True)],
)

# Evaluate the model on the test set
val_loss, val_acc = model_100.evaluate(X_val, y_val)

# Save the trained model
model_100.save("models/my_model.h5")


Epoch 1/100
4/4 [==============================] - 0s 55ms/step - loss: 0.4530 - mean_absolute_error: 0.5490 - val_loss: 0.4338 - val_mean_absolute_error: 0.5362
Epoch 2/100
4/4 [==============================] - 1s 176ms/step - loss: 0.4534 - mean_absolute_error: 0.5493 - val_loss: 0.4300 - val_mean_absolute_error: 0.5375
Epoch 3/100
4/4 [==============================] - 0s 26ms/step - loss: 0.4527 - mean_absolute_error: 0.5491 - val_loss: 0.4293 - val_mean_absolute_error: 0.5361
Epoch 4/100
4/4 [==============================] - 0s 26ms/step - loss: 0.4522 - mean_absolute_error: 0.5492 - val_loss: 0.4272 - val_mean_absolute_error: 0.5368
Epoch 5/100
4/4 [==============================] - 0s 25ms/step - loss: 0.4525 - mean_absolute_error: 0.5493 - val_loss: 0.4269 - val_mean_absolute_error: 0.5371
Epoch 6/100
4/4 [==============================] - 0s 20ms/step - loss: 0.4526 - mean_absolute_error: 0.5484 - val_loss: 0.4305 - val_mean_absolute_error: 0.5366
Epoch 7/100
4/4 [==========

<IPython.core.display.Javascript object>

In [46]:
from tensorflow.keras.models import load_model

# Load the saved model
saved_model = load_model('models/best_model.h5')



<IPython.core.display.Javascript object>

In [47]:
saved_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 32)                160       
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 1,282
Trainable params: 1,282
Non-trainable params: 0
_________________________________________________________________


<IPython.core.display.Javascript object>

In [49]:
# Evaluate the model on test data
test_loss, test_accuracy = saved_model.evaluate(X_train_100, y_train_100)

# Print the test accuracy
print("Test accuracy:", test_accuracy)
# Print the test accuracy
print("Test accuracy:", test_loss)

4/4 [==============================] - 0s 2ms/step - loss: 0.4531 - mean_absolute_error: 0.5488
Test accuracy: 0.5488085150718689
Test accuracy: 0.45308127999305725


<IPython.core.display.Javascript object>

In [51]:
history_extra.history

{'loss': [0.45302459597587585,
  0.45335841178894043,
  0.45265811681747437,
  0.45222967863082886,
  0.45247992873191833,
  0.4525514245033264,
  0.4529557526111603,
  0.45267462730407715,
  0.45274126529693604,
  0.45242801308631897,
  0.4525755047798157,
  0.45259588956832886,
  0.4533466398715973,
  0.45251235365867615,
  0.4525488018989563],
 'mean_absolute_error': [0.5490133166313171,
  0.5492874979972839,
  0.5491186380386353,
  0.549187958240509,
  0.5493232607841492,
  0.5484479069709778,
  0.5501682758331299,
  0.5488965511322021,
  0.5501302480697632,
  0.549655556678772,
  0.5485348701477051,
  0.5491210222244263,
  0.5488659143447876,
  0.5487604737281799,
  0.5492116808891296],
 'val_loss': [0.43375998735427856,
  0.42999568581581116,
  0.4292910397052765,
  0.4272305369377136,
  0.4268762767314911,
  0.430543452501297,
  0.42696598172187805,
  0.4304555058479309,
  0.43053168058395386,
  0.42871636152267456,
  0.4286343455314636,
  0.42745906114578247,
  0.42836743593215

<IPython.core.display.Javascript object>